# Assignment 03 - Due Friday, November 18 at 4pm

*Objectives*: Use Spark to process and perform basic analysis on non-relational data, including its DataFrame and SQL interfaces.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with Python code, SQL queries, shell commands, and markdown cells.  The notebook itself should be completely reproducible (using AWS EC2 instance based on the provided AMI) from start to finish; another person should be able to use the code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking when appropriate.


*Deadline*: Friday, November 18, 4 pm.  

## Part 1 - Setup

Begin by setting up Spark and fetching the project data.  

**Note**: you may want to use a larger EC2 instance type than normal.  This project was prepared using a `t2.xlarge` instance.  Just remember that the larger the instance, the higher the per-hour charge, so be sure to remember to shut your instance down when you're done, as always.

### About the data

We will use JSON data from Twitter; we saw an example of this in class.  It should parse cleanly, allowing you to focus on analysis.

This data was gathered using GWU Libraries' [Social Feed Manager](http://sfm.library.gwu.edu/) application during a recent game of the MLB World Series featuring the Los Angeles Dodgers and Houston Astros.  This first file tells you a little bit about how it was gathered:

#### First make sure you are working from the right working directory

In [93]:
!pwd

/home/ubuntu/notebooks


#### This below file provides context and detail information on the files we are going to work with
#### in this assignment

In [2]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt

--2022-11-13 17:33:49--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611-README.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.109.246
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.109.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1920 (1.9K) [text/plain]
Saving to: ‘9670f3399f774789b7c3e18975d25611-README.txt’

9670f3399f774789b7c 100%[===================>]   1.88K  --.-KB/s    in 0s      

2022-11-13 17:33:49 (97.8 MB/s) - ‘9670f3399f774789b7c3e18975d25611-README.txt’ saved [1920/1920]



In [3]:
!cat 9670f3399f774789b7c3e18975d25611-README.txt

This is an export created with Social Feed Manager.

EXPORT INFORMATION
Selected seeds: All seeds
Export id: 9670f3399f774789b7c3e18975d25611
Export type: twitter_filter
Format: Full JSON
Export completed:  Oct. 30, 2017, 11:21:04 p.m. EDT
Deduplicate: No

COLLECTION INFORMATION
Collection name: test set for world series
Collection id: 34e3f7460b5c4df09d64a1e61fd81238
Collection set: mlb-test (collection set id d6e8c27b1bc942e78790aa55a82b3a7a)
Harvest type: Twitter filter
Collection description: running for just one hour, just for fun.

Harvest options:
Media: No
Web resources: No

Seeds:
* Track: dodgers,astros - Active

Change log:

Change to test set for world series (collection) on Oct. 30, 2017, 10:59:56 p.m. EDT by dchud:
* is_active: "True" changed to "False"

Change to test set for world series (collection) on Oct. 30, 2017, 10:58:51 p.m. EDT by dchud:
* is_on: "True" changed to "False"

Change to test set for world series (collection) on Oct. 29, 2017, 8:01:24 p.m. EDT by dch

The most important pieces in that metadata are:

 * It tracked tweets that mentioned "dodgers" or "astros".  Every item in this set should refer to one or the other, or both.
 * This data was not deduplicated; we may see individual items more than once.
 * Data was collected between October 29 and October 30.  Game 5 of the Series was played during this time.
 
You should not need to know anything about baseball to complete this assignment.

**Please note**: sometimes social media data contains offensive material.  This data set has not been filtered; if you do come across something inappropriate, please do your best to ignore it if you can.

## Fetch the data

The following files are available:

 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json
 * https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
 
### Q1.1 - Upload the above files to your instance using `wget`.  Verify the file sizes using the command line. 

Each file should contain exactly 100,000 tweets.  

*Note*: you are required to use all files.  It will be easier to process more data if you use a larger EC2 instance type, as suggested above.  Use the exact same set of files throughout the assignment.

**Answer**

In [ ]:
# Upload the files using wget. Add more if needed.

In [42]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json

--2022-11-13 19:21:52--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_003.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.163.176
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.163.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595711407 (568M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_003.json’

9670f3399f774789b7c 100%[===================>] 568.11M  96.2MB/s    in 6.0s    

2022-11-13 19:21:58 (94.2 MB/s) - ‘9670f3399f774789b7c3e18975d25611_003.json’ saved [595711407/595711407]



In [43]:
!mv 9670f3399f774789b7c3e18975d25611_003.json tweet3.json

In [44]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json

--2022-11-13 19:22:07--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_004.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.76.150
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.76.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549995846 (525M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_004.json’

9670f3399f774789b7c 100%[===================>] 524.52M  61.0MB/s    in 8.9s    

2022-11-13 19:22:16 (58.9 MB/s) - ‘9670f3399f774789b7c3e18975d25611_004.json’ saved [549995846/549995846]



In [45]:
!mv 9670f3399f774789b7c3e18975d25611_004.json tweet4.json

In [46]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json

--2022-11-13 19:22:38--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_005.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.184.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.184.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530698683 (506M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_005.json’

9670f3399f774789b7c 100%[===================>] 506.11M  65.3MB/s    in 7.9s    

2022-11-13 19:22:46 (64.3 MB/s) - ‘9670f3399f774789b7c3e18975d25611_005.json’ saved [530698683/530698683]



In [47]:
!mv 9670f3399f774789b7c3e18975d25611_005.json tweet5.json

In [48]:
!wget https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json

--2022-11-13 19:23:01--  https://s3.amazonaws.com/2017-dmfa/project-3/9670f3399f774789b7c3e18975d25611_006.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.204.53
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.204.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 545081593 (520M) [application/json]
Saving to: ‘9670f3399f774789b7c3e18975d25611_006.json’

9670f3399f774789b7c 100%[===================>] 519.83M  61.7MB/s    in 8.8s    

2022-11-13 19:23:10 (58.9 MB/s) - ‘9670f3399f774789b7c3e18975d25611_006.json’ saved [545081593/545081593]



In [49]:
!mv 9670f3399f774789b7c3e18975d25611_006.json tweet6.json

#### Check the files have exactly 100,000 tweets using the command line

In [ ]:
# Write you code here. 

In [50]:
!wc -l tweet3.json

100000 tweet3.json


In [51]:
!wc -l tweet4.json

100000 tweet4.json


In [52]:
!wc -l tweet5.json

100000 tweet5.json


In [53]:
!wc -l tweet6.json

100000 tweet6.json


In [55]:
!wc -l tweet*.json

    100000 tweet3.json
    100000 tweet4.json
    100000 tweet5.json
    100000 tweet6.json
    400000 total


For your reference, here is the text of one Tweet, randomly selected from one of these files.  You might wish to study its structure and refer to it later.

In [56]:
!cat *.json | shuf -n 1 > example-tweet.json

In [57]:
import json
print(json.dumps(json.load(open("example-tweet.json")), indent=2))

{
  "quote_count": 0,
  "contributors": null,
  "truncated": false,
  "text": "RT @BBBaumgartner: Let's go. Let's go. Let's go. @Dodgers",
  "is_quote_status": false,
  "in_reply_to_status_id": null,
  "reply_count": 0,
  "id": 924856124699959296,
  "favorite_count": 0,
  "entities": {
    "user_mentions": [
      {
        "id": 26020351,
        "indices": [
          3,
          17
        ],
        "id_str": "26020351",
        "screen_name": "BBBaumgartner",
        "name": "Brian Baumgartner"
      },
      {
        "id": 23043294,
        "indices": [
          49,
          57
        ],
        "id_str": "23043294",
        "screen_name": "Dodgers",
        "name": "Los Angeles Dodgers"
      }
    ],
    "symbols": [],
    "hashtags": [],
    "urls": []
  },
  "retweeted": false,
  "coordinates": null,
  "timestamp_ms": "1509337857781",
  "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
  "in_reply_to_screen_name": null,
 

You can find several key elements in this example; the text, time, and language of the tweet, whether it was a reply to another user, the user's screen name along with their primary language and other account information like creation date, follower/friend/tweet counts, and perhaps their location.  
If there are hashtags, user mentions, or urls present in their tweet, they will be present in the `entities` section; these are not present in every tweet.  If this is a retweet, you will see the original tweet and its information nested within.

### Q1.2 - Start up Spark, and verify the file sizes.

We will use our normal startup sequence here:

In [1]:
import os

In [2]:
os.environ['SPARK_HOME'] = '/usr/local/lib/spark'

In [3]:
import findspark

In [4]:
findspark.init()

In [5]:
from pyspark import SparkContext

In [6]:
spark = SparkContext(appName='project-03')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 14:36:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark

<SparkContext master=local[*] appName=project-03>

In [8]:
from pyspark import SQLContext

In [9]:
sqlc = SQLContext(spark)

/usr/local/lib/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [10]:
sqlc

In [14]:
tweets = sqlc.read.json("*.json")

Verify that Spark has loaded the same number of tweets you saw before:

**Answer**

In [11]:
tweets = sqlc.read.json(["tweet3.json", "tweet4.json", "tweet5.json" , "tweet6.json"])

22/11/18 14:36:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
tweets.count()

400000

Do you see exactly the same number of tweets in Spark that you saw on the command line?

## Yes, the rows are same.

In [14]:
tweets.columns

['contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user']

In [15]:
tweets.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [13]:
tweets.take(2)

[Row(contributors=None, coordinates=None, created_at='Mon Oct 30 08:18:55 +0000 2017', display_text_range=None, entities=Row(hashtags=[], media=None, symbols=[], urls=[Row(display_url='twitter.com/GallantSays/st…', expanded_url='https://twitter.com/GallantSays/status/924877135579402240', indices=[26, 49], url='https://t.co/MtcRbtuxhc')], user_mentions=[Row(id=25950370, id_str='25950370', indices=[3, 19], name='Sean Pendergast', screen_name='SeanTPendergast')]), extended_entities=None, extended_tweet=None, favorite_count=0, favorited=False, filter_level='low', geo=None, id=924913493261934592, id_str='924913493261934592', in_reply_to_screen_name=None, in_reply_to_status_id=None, in_reply_to_status_id_str=None, in_reply_to_user_id=None, in_reply_to_user_id_str=None, is_quote_status=True, lang='en', place=None, possibly_sensitive=False, quote_count=0, quoted_status=Row(contributors=None, coordinates=None, created_at='Mon Oct 30 05:54:27 +0000 2017', display_text_range=None, entities=Row(ha

## Part 2 - Comparing DataFrames and Spark SQL

For the next three questions, we will look at operations using both DataFrames and SQL queries. Note that `tweets` is already a DataFrame:

To issue SQL queries, we need to register a table based on `tweets`:

This is all well and good, but how well did schema inference work?

To issue SQL queries, we need to register a table based on tweets:

In [49]:
tweets.createOrReplaceTempView("tweets")

### Q2.1 - Which 10 languages are most commonly used in tweets?  Verify your result by executing it with both the dataframe and with SQL.

Hint: for the dataframe, use `groupBy`, `count`, and `orderBy`.  See the documentation at https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html for details on these and other functions.

**Answer**

In [ ]:
# Edit this cell and write query using dataframe

In [55]:
tweets.groupBy("lang").count().orderBy("count", ascending=False).show(10)

+----+------+
|lang| count|
+----+------+
|  en|346302|
|  es| 31869|
| und| 14659|
|  in|  2355|
|  fr|   997|
|  pt|   914|
|  tl|   485|
|  ht|   377|
|  nl|   347|
|  it|   285|
+----+------+
only showing top 10 rows



In [ ]:
# Edit this cell and write query using SQL

In [18]:
sqlc.sql("""
         SELECT lang, 
         COUNT(*) As Count 
         FROM tweets 
         GROUP BY lang 
         ORDER BY count DESC
""").show(10)


+----+------+
|lang| Count|
+----+------+
|  en|346302|
|  es| 31869|
| und| 14659|
|  in|  2355|
|  fr|   997|
|  pt|   914|
|  tl|   485|
|  ht|   377|
|  nl|   347|
|  it|   285|
+----+------+
only showing top 10 rows



### Q2.2 - Which 10 time zones are most common among users?  Verify your result with both the dataframe and SQL.

*Note*: for this question, you may leave NULL values present in your results, as a way to help you understand what data is present and what is missing.

**Answer**

In [ ]:
# Edit this cell and write query using dataframe

In [53]:
tweets.groupBy("user.time_zone").count().orderBy("count", ascending=False).show(10)

+--------------------+------+
|           time_zone| count|
+--------------------+------+
|                null|166386|
|Pacific Time (US ...| 72214|
|Central Time (US ...| 63664|
|Eastern Time (US ...| 35889|
|             Arizona| 10748|
|Mountain Time (US...| 10057|
|Atlantic Time (Ca...|  4805|
|               Quito|  4397|
|             Caracas|  4218|
|         Mexico City|  3935|
+--------------------+------+
only showing top 10 rows



In [ ]:
# Edit this cell and write query using SQL

In [58]:
sqlc.sql("""
         SELECT user.time_zone, 
         COUNT(*) As Count 
         FROM tweets 
         GROUP BY user.time_zone
         ORDER BY count DESC
""").show(10)

+--------------------+------+
|           time_zone| Count|
+--------------------+------+
|                null|166386|
|Pacific Time (US ...| 72214|
|Central Time (US ...| 63664|
|Eastern Time (US ...| 35889|
|             Arizona| 10748|
|Mountain Time (US...| 10057|
|Atlantic Time (Ca...|  4805|
|               Quito|  4397|
|             Caracas|  4218|
|         Mexico City|  3935|
+--------------------+------+
only showing top 10 rows



### Q2.3 - How many tweets mention the Dodgers?  How many mention the Astros?  How many mention both?

You may use either the dataframe or SQL to answer.  Explain why you have chosen that approach.

Hint:  you will want to look at the value of the `text` field.

**Answer**

In [18]:
# Edit this cell and add more as appropriate
sqlc.sql("""
         SELECT count(*) as Dodgers_Tweets
         FROM tweets 
         where UPPER(text) LIKE UPPER('%Dodgers%')
""").show()

+--------------+
|Dodgers_Tweets|
+--------------+
|        168218|
+--------------+



In [16]:
sqlc.sql("""
         SELECT count(*) as Astros_Tweets
         FROM tweets 
         where upper(text) LIKE upper('%Astros%')
""").show()

+-------------+
|Astros_Tweets|
+-------------+
|       246465|
+-------------+



In [17]:
sqlc.sql("""
         SELECT count(*) as Both_Tweets
         FROM tweets 
         where upper(text) LIKE upper('%Dodgers%')
         and upper(text) LIKE upper('%Astros%')
""").show()

+-----------+
|Both_Tweets|
+-----------+
|      49545|
+-----------+



## Part 3 - More complex queries

For this section, you may choose to use dataframe queries or SQL.  If you wish, you may verify results by using both, as in Part 2, but this is not required for this section.

### Q3.1 - Team mentions by location

In which users' locations are the Astros and the Dodgers being mentioned the most?  Consider each team separately, one at a time.  Discuss your findings. Do not count null time_zones or location.

Hint:  you may use either the time zones or user-specified locations for this question.


**Answer**

In [17]:
sqlc.sql("""
         SELECT user.location, count(*) as Tweet_count
         FROM tweets 
         where upper(text) LIKE upper('%Dodgers%')
         and user.location is not null
         group by user.location
         order by Tweet_count desc
""").show(10)

+---------------+-----------+
|       location|Tweet_count|
+---------------+-----------+
|Los Angeles, CA|       6361|
|California, USA|       2701|
|    Houston, TX|       2362|
|    Los Angeles|       1967|
|  United States|       1578|
|     California|        866|
|     Texas, USA|        832|
|         México|        698|
|  Las Vegas, NV|        610|
|      Venezuela|        607|
+---------------+-----------+
only showing top 10 rows



In [19]:
sqlc.sql("""
         SELECT user.location, count(*) as Tweet_count
         FROM tweets 
         where upper(text) LIKE upper('%Astros%')
         and user.location is not null
         group by user.location
         order by Tweet_count desc
""").show(10)

+---------------+-----------+
|       location|Tweet_count|
+---------------+-----------+
|    Houston, TX|      17639|
|     Texas, USA|       4159|
|          Texas|       2647|
|Los Angeles, CA|       2448|
|        Houston|       2133|
|  United States|       1855|
| Houston, Texas|       1852|
|     Austin, TX|       1503|
|California, USA|       1294|
|San Antonio, TX|       1269|
+---------------+-----------+
only showing top 10 rows



### Q3.2 - Which Twitter users are being replied to the most?

Discuss your findings.

Hint: use the top-level `in_reply_to_screen_name` for this.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate

In [23]:
sqlc.sql("""
         SELECT in_reply_to_screen_name, count(*) as Reply_Cnt
         FROM tweets 
         where in_reply_to_screen_name is not null
         group by in_reply_to_screen_name
         order by Reply_Cnt desc
         
""").show(10)

+-----------------------+---------+
|in_reply_to_screen_name|Reply_Cnt|
+-----------------------+---------+
|                 astros|     4034|
|                Dodgers|     3297|
|                    MLB|      705|
|          stephenasmith|      332|
|               MLBONFOX|      217|
|              adevaldes|      180|
|          DodgersNation|      165|
|          rolandsmartin|      141|
|        JustinVerlander|      118|
|          DodgerInsider|      113|
+-----------------------+---------+
only showing top 10 rows



Wrtie your observations here


### Q3.3 - Which 10 verified users have the most followers?  Which 10 unverified users have the most followers?

Provide both the screen names (screen_name) and follower counts (followers_count) for each.
Verified users -- use verified == 't'

Discuss your findings.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate

In [25]:
sqlc.sql("""
        select user.screen_name,user.verified, max(user.followers_count) as max_follower_cnt 
        from tweets
        where user.verified == 't'
        group by user.screen_name,user.verified
        order by max_follower_cnt desc
""").show(10)

+--------------+--------+----------------+
|   screen_name|verified|max_follower_cnt|
+--------------+--------+----------------+
|        cnnbrk|    true|        53191119|
|       nytimes|    true|        39959480|
|           CNN|    true|        38209973|
|       Reuters|    true|        18937529|
|       FoxNews|    true|        16272836|
|           ABC|    true|        12551437|
|washingtonpost|    true|        11417638|
|   lopezdoriga|    true|         7859821|
|           MLB|    true|         7841255|
|           NPR|    true|         7289619|
+--------------+--------+----------------+
only showing top 10 rows



In [26]:
sqlc.sql("""
        select user.screen_name,user.verified, max(user.followers_count) as max_follower_cnt 
        from tweets
        where user.verified == 'f'
        group by user.screen_name,user.verified
        order by max_follower_cnt desc
""").show(10)

+---------------+--------+----------------+
|    screen_name|verified|max_follower_cnt|
+---------------+--------+----------------+
|Daminous_Purity|   false|          998742|
|        chochos|   false|          833669|
|     TexasHumor|   false|          816301|
|  el_carabobeno|   false|          725952|
|       PAMsLOvE|   false|          712254|
| mlbtraderumors|   false|          659851|
|        jilevin|   false|          568341|
|    sun_das_ill|   false|          559669|
|   DiegoArcos14|   false|          544926|
|    TVCDeportes|   false|          543095|
+---------------+--------+----------------+
only showing top 10 rows



### Q4 - Analyze common words in tweet text

Following the example in class, use `tweets.rdd` to find the most common interesting words in tweet text.  To keep it "interesting", remove at least 10 common stop words found in tweets, like "a", "an", "the", and "RT" (you might want to derive these stop words from initial results).  A simple split on text whitespace like we had in class is sufficient; you do not have to account for punctuation. 

After you find the most common words, use dataframe or SQL queries to find patterns among how those words are used.  For example, are they more frequently used by Dodgers or Astros fans, or by people in one part of the country over another?  Explore and see what you can find, and discuss your findings.

You will notice that common words include words like "thisteam" and "earnhistory". I would like you to write two queries to investigate whether those two words are used by the Astros or Dodgers

Hint: don't forget all the word count pipeline steps we used earlier in class.

**Answer**

In [ ]:
# Edit this cell and add more as appropriate ## use below code

In [ ]:
# Removing the stop word and finding the most common words using rdd.

In [96]:
stop = ['', 'the', 'and', 'of', 'is', 'in', 'a', 'it', 'to', 'as', 'by', 'with',
        'or', 'its', 'from', 'at', 'rt','this','for','i','are','on','you']
tweets.rdd.flatMap(lambda x: x['text'].split(' '))\
    .map(lambda x: (x.lower(), 1))\
    .reduceByKey(lambda x, y: x+y)\
    .filter(lambda x: x[0] not in stop)\
    .top(15, lambda x: x[1])

[('astros', 88985),
 ('dodgers', 70845),
 ('#worldseries', 55042),
 ('@astros:', 49823),
 ('game', 45437),
 ('@astros', 40051),
 ('#thisteam', 38429),
 ('#earnhistory', 37293),
 ('#astros', 31108),
 ('@dodgers', 29941),
 ('go', 27423),
 ('that', 25560),
 ('#dodgers', 23726),
 ('win', 23221),
 ('@dodgers:', 22143)]

In [56]:
#Second Part- Checking for common word "win" with location
sqlc.sql("""
         Select user.location, count(*) as count       
         FROM tweets
         where upper(text) LIKE upper('%win%')
         and user.location is not null
         group by user.location
         order by count desc
""").show(10) 

+---------------+-----+
|       location|count|
+---------------+-----+
|    Houston, TX| 3178|
|Los Angeles, CA|  741|
|     Texas, USA|  694|
|          Texas|  406|
|  United States|  380|
|California, USA|  364|
|        Houston|  338|
| Houston, Texas|  299|
|     Austin, TX|  240|
|San Antonio, TX|  211|
+---------------+-----+
only showing top 10 rows



In [ ]:
#Second Part- Checking for common word "#thisteam" with location

In [57]:
sqlc.sql("""
         Select user.location, count(*) as count       
         FROM tweets
         where upper(text) LIKE upper('%#thisteam%')
         and user.location is not null
         group by user.location
         order by count desc
""").show(10)

+-------------------+-----+
|           location|count|
+-------------------+-----+
|    Los Angeles, CA| 1582|
|    California, USA|  621|
|        Los Angeles|  448|
|         California|  201|
|      United States|  160|
|Southern California|  157|
|             México|  129|
|      Las Vegas, NV|  107|
|                 LA|  104|
|     Long Beach, CA|   92|
+-------------------+-----+
only showing top 10 rows



In [ ]:
#Second Part- Checking for common word "%#earnhistory%" with location

In [58]:
sqlc.sql("""
         Select user.location, count(*) as count       
         FROM tweets
         where upper(text) LIKE upper('%#earnhistory%')
         and user.location is not null
         group by user.location
         order by count desc
""").show(10)

+---------------+-----+
|       location|count|
+---------------+-----+
|    Houston, TX| 6091|
|     Texas, USA| 1167|
|          Texas|  796|
| Houston, Texas|  648|
|        Houston|  597|
|     Austin, TX|  386|
|San Antonio, TX|  311|
|            HTX|  286|
|  United States|  264|
|    Houston, Tx|  263|
+---------------+-----+
only showing top 10 rows



### Third Part
## Here I am trying to show the different ways of how the common word Earn History and This Team is associated with Astros and Dodgers 

In [91]:
# First using screen-names against #thisteam.
sqlc.sql("""
         SELECT user.screen_name, count(*) as count
         FROM tweets 
         where upper(text) LIKE upper('%#thisteam%') 
         and (user.screen_name iLIKE '%Dodgers%' or user.screen_name iLIKE '%Astros%')
         group by user.screen_name
         order by count desc
""").show(15) 

+---------------+-----+
|    screen_name|count|
+---------------+-----+
|   DodgersWayFS|   32|
|   DodgersViews|   12|
|LADodgersAllDay|    6|
|      34Dodgers|    6|
|  OccupyDodgers|    6|
|     DodgersPTY|    6|
|        Dodgers|    5|
|    Dodgers2915|    5|
|     LAxDodgers|    4|
|  DodgersNation|    4|
|  DodgersLakers|    4|
|    DodgersShoe|    4|
|  gododgers1317|    3|
| UpdatesDodgers|    3|
|luvthedodgers23|    2|
+---------------+-----+
only showing top 15 rows



In [92]:
# using screen-names against #earnhistory
sqlc.sql("""
         SELECT user.screen_name, count(*) as count
         FROM tweets 
         where upper(text) LIKE upper('%#earnhistory%') 
         and (user.screen_name iLIKE '%Dodgers%' or user.screen_name iLIKE '%Astros%')
         group by user.screen_name
         order by count desc
""").show(15) ##check it 

+---------------+-----+
|    screen_name|count|
+---------------+-----+
|         astros|   19|
| CrazyAstrosFan|   14|
| Rocketexastros|   11|
|   AstrosBatman|    6|
|      Astros290|    5|
|  AstrosPower25|    5|
|    AstrosDaily|    4|
| Astros_Arsenal|    4|
|       astros76|    4|
| AstrosTrainGuy|    4|
|      Astros365|    4|
|   AstrosFrance|    4|
|AstrosCrushGirl|    4|
|  astros_center|    3|
|    AstrosFan42|    3|
+---------------+-----+
only showing top 15 rows



In [95]:
#Part three using analysis of which word combinations are used more frequently in tweets
sqlc.sql("""
         select sum(Dodgers_earnhistory), sum(Dodgers_thisteam),
         sum(Astros_earnhistory),sum(Astros_thisteam)
         from
         (Select text,        
         IF((upper(text) LIKE upper('%Dodgers%')) and (upper(text) LIKE upper('%#earnhistory%')) and 
         (upper(text) NOT LIKE upper('%astros%')) ,1,0) as Dodgers_earnhistory,
         IF((upper(text) LIKE upper('%Dodgers%')) and (upper(text) LIKE upper('%#thisteam%')) and 
         (upper(text) NOT LIKE upper('%astros%')),1,0) as Dodgers_thisteam,
         IF((upper(text) LIKE upper('%Astros%')) and (upper(text) LIKE upper('%#earnhistory%'))and 
         (upper(text) NOT LIKE upper('%dodgers%')),1,0) as Astros_earnhistory,
         IF((upper(text) LIKE upper('%Astros%')) and (upper(text) LIKE upper('%#thisteam%'))and 
         (upper(text) NOT LIKE upper('%dodgers%')),1,0) as Astros_thisteam
         FROM tweets
         group by text)
""").show(10) 

+------------------------+---------------------+-----------------------+--------------------+
|sum(Dodgers_earnhistory)|sum(Dodgers_thisteam)|sum(Astros_earnhistory)|sum(Astros_thisteam)|
+------------------------+---------------------+-----------------------+--------------------+
|                     290|                 5808|                   6972|                 467|
+------------------------+---------------------+-----------------------+--------------------+



In [ ]:
Observation: Dodgers used thisteam more and Astros used earnhistory more.

### Final Conclusion:
** After observing the common word(#earnhistory and #thisteam) associated with Astros and Dodgers, first with screen names and then doing a generalized query. We have found that #Earnhistory is used by Astros more and #thisteam is used by Dodgers more.** 
